Import package dependencies including pandas and numpy for data manipulation and analysis, sklearn for modelling and coefficient analysis, and plotly for graphing our results

## Crisp-DM Step 1) Understand the Business  
In this scenario, Airbnb is a company that allows property owners to rent out their residential properties much like a hotel or bed and breakfast would.  
Our goal for this business scenario is to provide an analysis for a potential property investor in the Seattle area to have the best chance of success in the Airbnb rental market, and to help them make decisions about what property to invest in and how to handle it.

Question 1) Which zip codes in the Seattle area are associated with higher apartment rent prices?

Question 2) Does "super host" status influence allow an owner to charge more for their apartment, and should the investor try and become one?  

Question 3) What are some of the key contributing factors that "super hosts" have, and what should our investor aim to do to become one?  

In [32]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## CRISP-DM Step 2) Data Understanding
Bringing in the key datasets provided for our analysis.  
Data Sets include:  
df_cal (the .csv of calendar price/availability for listings)  
df_list (the .csv of many columns related to a listing including its physical characteristics as well as host information)  
df_rev (the .csv of user reviews of the airbnbs/hosts)

In [33]:
df_cal = pd.read_csv(r'C:\Users\AMELATTI\Documents\GitHub\project1_seattle_airbnb\data\calendar.csv')
df_list = pd.read_csv(r'C:\Users\AMELATTI\Documents\GitHub\project1_seattle_airbnb\data\listings.csv')
df_rev = pd.read_csv(r'C:\Users\AMELATTI\Documents\GitHub\project1_seattle_airbnb\data\reviews.csv')

Checking the shapes of our datasets to get an understanding of the number of rows of data and the number of columns to look through for potential analysis.  
Also looking at column headers to understand what data may be present.

In [34]:
print(df_cal.shape)
print(df_list.shape)
print(df_rev.shape)

print(df_cal.columns.tolist())
print(df_list.columns.tolist())
print(df_rev.columns.tolist())


(1393570, 4)
(3818, 92)
(84849, 6)
['listing_id', 'date', 'available', 'price']
['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'pr

## Question 1) Which zip codes in the Seattle area are associated with higher apartment rent prices?
We will address this by:
1) Extracting properties with the apartment designation
2) Selecting a subset of apartments with a ~~size of 600-800 sq.ft~~  
    a) Only ~3% of all properties had square feet populated, so opted for metrics using # of bed and bath (1 and 1 for apts)  
    b) This prevents outlier apartments from potentially influencing prices (a mega apartment with 4 bed 4 bath as an example)
3) Cleaning data by removing listings that are missing size or bed/bath information
4) Plotting the median apartment prices vs. zip code and taking the top 5 zip codes with highest median apartment price.

## Crisp-DM Step 3) Preparation of Data for Question 1  
To answer Question 1, we need to perform question steps 1,2, and 3 via filtering, cleansing, and reformatting the data in df_list.

In [35]:
# Check initial shape of df_list
print('# of total properties = {}'.format(df_list.shape[0]))

# Filter to apartments only via column 'property type', and see how many apartments we have.
df_apt = df_list[df_list['property_type'] == 'Apartment']
print('# of total apartments = {}'.format(df_apt.shape[0]))

# Filter to only apartments with 1 bed and 1 bath, as the square footage column was barely populated. Check how many remain.
df_apt_sized = df_apt[(df_apt['bedrooms'] == 1) & (df_apt['bathrooms'] == 1)]
print('# of total apartments with 1 bed and 1 bath = {}'.format(df_apt_sized.shape[0]))

# Cleaning data set by dropping any rows that have an NaN in critical columns of id, price, and zipcode.
df_apt_final = df_apt_sized.dropna(subset=['id','price','zipcode'], axis=0)
print('# of valid data rows with no NaNs for id/price/zip = {}'.format(df_apt_final.shape[0]))

# Converting price, which is a string, to a float value that we can obtain the median of. Had to remove the $ sign to allow .astype to work!
df_apt_final['price'] = df_apt_final['price'].str.replace('$','').astype(float)

# To determine prices by zipcode, group the filtered and cleaned dataframe by zipcode and then aggregate the median price and number of apartments included.
df_apt_final_zip = df_apt_final.groupby(by=['zipcode']).agg(
        median_price = ('price', 'median'),
        apt_count = ('id', 'count')

).reset_index()

# Lastly, sort the rows by median price so that the most expensive zip codes are at the top.
df_apt_final_zip = df_apt_final_zip.sort_values('median_price')


# of total properties = 3818
# of total apartments = 1708
# of total apartments with 1 bed and 1 bath = 1063
# of valid data rows with no NaNs for id/price/zip = 1063


C:\Users\AMELATTI\AppData\Local\Temp\ipykernel_20264\1886165759.py:17: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



## Crisp-DM Step 5) Data Evaluation for Question 1
After cleaning the data set, it is filtered one last time (only count zipcodes with 5 or more apartments) to then be plotted in a plotly go chart.  
This plotly go chart includes a bar chart of the number of apartments in each zip code that were used for the analysis. (y axis)  
It then also includes a scatter plot of the median price for apartments in each zip code, which is what we wanted to analyze. (y2 axis)  
The zip codes with the highest median daily rent price of apartments in the seattle area are clearly displayed on the right hand of the graph

In [36]:
# In order to avoid skewing median price if only a few apartments are in a zip code, I chose to remove zip code data points that had less than 5 apartments listed.
print('Start with {} apartments over {} zip codes'.format(df_apt_final_zip['apt_count'].sum(), df_apt_final_zip['zipcode'].count(),))
df_apt_final_zip = df_apt_final_zip[df_apt_final_zip['apt_count'] >= 5]
print('After removing zip codes with < 5 apartments, we will only plot {} apartments over {} zip codes'.format(df_apt_final_zip['apt_count'].sum(), df_apt_final_zip['zipcode'].count(),))

# Plot of median prices for 1 bed 1 bath apartments by zipcode, along with counts of apartments included
fig_counts = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart of the number of apartments in each zip code that were used for the analysis. (y axis)  
fig_counts.add_trace(
        go.Bar(name='Total Counts of Apartments Per Zip Code', x=df_apt_final_zip['zipcode'], y=df_apt_final_zip['apt_count'],
        yaxis='y',
        )),

# Scatter plot of the median price for apartments in each zip code, which is what we wanted to analyze. (y2 axis) 
fig_counts.add_trace(
        go.Scatter(name='Median Price of Apartments Per Zip Code', x=df_apt_final_zip['zipcode'], y=df_apt_final_zip['median_price'],
        yaxis='y2',
        mode='lines+markers+text', legendrank=2, text=df_apt_final_zip['median_price'], 
        textposition= "top right",
        texttemplate = "%{y:$.0f}"
        ))

# Adding in titles and formatting
fig_counts.update_layout(
                    title = ("Median Prices of 1Bd/1Ba Seattle Airbnb Apartments by Zip Code with Counts of Apartments for Comparison <br>Data Includes {:,} 1 Bd/1Ba Apartments Posted for {} Zipcodes on Seattle Airbnb <br>Data Timeframe between {} and {}".format(
                        df_apt_final_zip['apt_count'].sum(),
                        df_apt_final_zip['zipcode'].count(),
                        min(df_cal['date']),
                        max(df_cal['date'])
                        )
                        ),
                    xaxis_title="Zipcode",
                    yaxis_title="Count of Apartments",
                    yaxis2_title="Median Price of Apartments",
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=0.01
                        )
                    )
fig_counts.update_layout(yaxis2_tickprefix = '$')
fig_counts.show()

Start with 1063 apartments over 26 zip codes
After removing zip codes with < 5 apartments, we will only plot 1054 apartments over 21 zip codes


## Crisp-DM Step 6) Final information + Conclusion for Question 1
Question 1 Takeaways for Investor: Top 5 Seattle Airbnb Zip Codes for 1bd/1ba apartment rent by median price are ['98101', '98121', '98104', '98199', '98119'].  
If the investor was looking to buy an apartment that would have the highest median daily rent price, they would look no further than these zip codes!  

And of course, daily rent price can be influenced by a ton of different factors such as amenities, location, peak tourist times, and even the owner’s own valuation of their rent!

In [37]:
# Get a dataframe of the 5 most expensive zip codes (as they've already been sorted in the previous df, we can use .tail)
df_top_5 = df_apt_final_zip[['zipcode','median_price','apt_count']].tail(5).sort_values('median_price',ascending=False)

print(df_top_5['zipcode'].to_list())
list_top_5 = df_top_5['zipcode'].to_list()

['98101', '98121', '98104', '98199', '98119']


## Crisp-DM Step 3) Preparation of Data for Question 2 
Question 2) Does "super host" status influence allow an owner to charge more for their apartment, and should the investor try and become one?  
1) Reuse df_apt_final and the top five zip codes in price from Question 1 because they contain our desired subset of apartments for the investor.
2) Ensure that we group by zipcodes again, as it wouldn't be fair to compare prices of superhosts from a cheaper zipcode to another more expensive zipcode.
3) Plot out median prices in each zip code with a designation of host is super host or not.

In [38]:
# Reuse original filtered apt dataframe but only a select few columns
df_apt_final_host = df_apt_final[['id','zipcode','price','host_is_superhost','host_id']]

# Using list_top_5 of the 5 most expensive zipcodes to filter to those 5 in our df
df_apt_final_host = df_apt_final_host[df_apt_final_host['zipcode'].isin(list_top_5)]

# Setting up a column for host differentiation for individual colors in the plots
df_apt_final_host['color_ind'] = np.where(df_apt_final_host['host_is_superhost'] == 't','red','gray')

df_apt_final_host.head()

,id,zipcode,price,host_is_superhost,host_id,color_ind
0,241032,98119,85.0,f,956883,gray
1,953595,98119,150.0,t,5177328,red
9,2493658,98119,150.0,t,5177328,red
16,3245876,98119,66.0,f,9465817,gray
29,4520099,98119,80.0,f,6184895,gray


In [39]:
# Group the Median Values by zipcode to include on plot to present clearer view rather than multiple smaller data points per zipcode!
# Aggregate by median price based on the super host or not super host status, and make sure the color indicator stays with our values.

df_apt_final_host_grouped = df_apt_final_host.groupby(by=['zipcode','host_is_superhost']).agg(
        median_price_host = ('price','median'),
        color_ind = ('color_ind','last')

).reset_index()

df_apt_final_host_grouped.head()

,zipcode,host_is_superhost,median_price_host,color_ind
0,98101,f,140.0,gray
1,98101,t,150.0,red
2,98104,f,128.0,gray
3,98104,t,132.0,red
4,98119,f,112.0,gray


## Crisp-DM Step 5) Data Evaluation for Question 2
Here we'll use another plotly go chart.  
This plotly go chart includes two scatter plots:  
Plot one (smaller dots) is each of the daily rent prices of the individual apartments in each of the top five most expensive Seattle zip codes.  
Plot two (larger dots with outlines) is the median of each of the daily rent prices of the individual apartments in each of the top five most expensive Seattle zip codes.  
This ensures easy visualization of the resulting difference in median rent price based on super host status.  

In [40]:
# Plot of median prices for 1 bed 1 bath apartments by zipcode, along with counts of apartments included
fig_hosts = make_subplots(specs=[[{"secondary_y": True}]])

# Plot one (smaller dots) is each of the daily rent prices of the individual apartments in each of the top five most expensive Seattle zip codes.  
fig_hosts.add_trace(
        go.Scatter(name='Daily Rent Price of Apartments Per Zip Code by Super Host Status', x=df_apt_final_host['zipcode'], y=df_apt_final_host['price'],
        yaxis='y',
        mode='markers',
        marker=dict(color=df_apt_final_host['color_ind'])
        ))

# Plot two (larger dots with outlines) is the median of each of the daily rent prices of the individual apartments in each of the top five most expensive Seattle zip codes.       
fig_hosts.add_trace(
        go.Scatter(name='Median Daily Rent Price of Apartments Per Zip Code by Super Host Status', x=df_apt_final_host_grouped['zipcode'], y=df_apt_final_host_grouped['median_price_host'],
        yaxis='y',
        mode='markers+text',
        marker=dict(size=14,
            color=df_apt_final_host_grouped['color_ind'],
            line=dict(width=2,color='Black')
            ),
        textposition= "middle right",
        texttemplate = "%{y:$.0f}",
        ))

# Adding in titles and formatting
fig_hosts.update_layout(
                    title = ("Daily Rent Price Analysis of 1Bd/1Ba Seattle Airbnb Apartments in Top Five Most Expensive Zip Codes vs. Super Host Status <br>Data Includes {:,} 1 Bd/1Ba Apartments Posted for {} Zipcodes on Seattle Airbnb <br>Data Timeframe between {} and {}".format(
                        df_apt_final_host['id'].count(),
                        df_apt_final_host['zipcode'].nunique(),
                        min(df_cal['date']),
                        max(df_cal['date'])
                        )
                        ),
                    xaxis_title="Zipcode",
                    yaxis_title="Daily Rent Price of Apartments",
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="right",
                        x=0.99
                        )
                    )
fig_hosts.update_layout(yaxis_tickprefix = '$')
fig_hosts.show()

## Crisp-DM Step 6) Final information + Conclusion for Question 2
Question 2 Takeaways for Investor: Apartment Rent Median Price Average Increase in the top 5 most expensive zipcodes.  
Being a super host shows an average increase in median daily rent price in the top 5 most expensive zip codes of the data!  
That % increase is printed below.  
From an outside perspective, if someone was a “super host” it is likely that they have a much better chance at successfully charging higher rent prices for their properties, getting repeat customers, and receiving additional foot traffic to their Airbnb page if they are ranked higher up in Airbnb’s search engine.

In [41]:
# Derive % expected increase in median rent value for super host status in top 5 most expensive zip codes

super_hosts_medians = df_apt_final_host_grouped['median_price_host'][df_apt_final_host_grouped['host_is_superhost'] == 't'].reset_index()
not_super_hosts_medians = df_apt_final_host_grouped['median_price_host'][df_apt_final_host_grouped['host_is_superhost'] == 'f'].reset_index()
percent_delta_medians = round(((super_hosts_medians / not_super_hosts_medians) - 1) * 100,2)

print('Super hosts see an average increase in median daily rent price across the top five zip codes of {}% vs. non-super hosts'.format(round(np.mean(percent_delta_medians)[1],2)))

Super hosts see an average increase in median daily rent price across the top five zip codes of 19.44% vs. non-super hosts


c:\Users\AMELATTI\AppData\Local\Ford-Conda\envs\py38_jupyter\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning:

In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'



## Crisp-DM Step 3) Preparation of Data for Question 3
Question 3) What are some of the key contributing factors that "super hosts" have, and what should our investor aim to do to become one?  
1) Pull as many columns that relate to being a "super host" as possible from df_list
2) Modify data to fit our comparison metrics
3) Determine the difference in percent of the averages for the columns between super hosts and not super hosts
4) Plot the graph to show the key elements that separate a super host from a not super host

In [42]:
# Creating dataframe df_super that includes a selection of columns from df_list that relate to a host's information and status
df_super = df_list[['id','host_is_superhost','host_id','host_response_time',
'host_response_rate',
'host_acceptance_rate',
'review_scores_communication',
'host_has_profile_pic',
'host_identity_verified',
'host_location',
'host_about']]
print('df_super original shape {}'.format(df_super.shape))

# Drop rows from these key columns with any missing data, EXCEPT FOR 
df_super = df_super.dropna(subset=['host_is_superhost','host_response_time','host_response_rate','host_acceptance_rate','review_scores_communication','host_location'],how='any',axis=0)
print('df_super shape after dropping key na rows is {}'.format(df_super.shape))

# host_about column converted to a boolean depending on whether or not the host included an about section for themselves
df_super['host_about_bool'] = np.where(~df_super['host_about'].isna(),True,False)

# Convert host_response_time to a consistent categorical value for use in linear regression model
df_super['host_response_time_int'] = np.where(df_super['host_response_time'] == 'a few days or more', 4, np.NaN)
df_super['host_response_time_int'] = np.where(df_super['host_response_time'] == 'within a day', 3, df_super['host_response_time_int'])
df_super['host_response_time_int'] = np.where(df_super['host_response_time'] == 'within a few hours', 2, df_super['host_response_time_int'])
df_super['host_response_time_int'] = np.where(df_super['host_response_time'] == 'within an hour', 1, df_super['host_response_time_int'])

# Convert host_response_rate to a quantitative value from a string
df_super['host_response_rate'] = df_super['host_response_rate'].str.replace('%','').astype(int)

# Convert multiple columns to bools for use in linear regression model.
df_super['host_is_superhost'] = np.where(df_super['host_is_superhost'] == 't',True,False)
df_super['host_has_profile_pic'] = np.where(df_super['host_has_profile_pic'] == 't',True,False)
df_super['host_identity_verified'] = np.where(df_super['host_identity_verified'] == 't',True,False)


# Convert host location from string values to numbers based on 0 = not in washington or seattle, 1= in washington, 2=in seattle
df_super['host_location'] = np.where(df_super['host_location'].str.contains('Seattle'),'Seattle',df_super['host_location'])
df_super['host_location'] = np.where(df_super['host_location'].str.contains('Washington'),'Wash',df_super['host_location'])
df_super['host_location'] = np.where((df_super['host_location'] != 'Seattle') & (df_super['host_location'] != 'Wash'),'Remote',df_super['host_location'])

df_super['host_location'] = np.where(df_super['host_location'] == 'Remote',0,df_super['host_location'])
df_super['host_location'] = np.where(df_super['host_location'] == 'Wash',1,df_super['host_location'])
df_super['host_location'] = np.where(df_super['host_location'] == 'Seattle',2,df_super['host_location'])

# Deleting columns that a) aren't included in the correlation, or b) is host_acceptance_rate which is 100% for ALL BUT ONE of the rows and would not provide meaningful results
df_super_clean = df_super.drop(columns=['id','host_id','host_response_time','host_about','host_acceptance_rate'])

df_super_clean.head()

df_super original shape (3818, 11)
df_super shape after dropping key na rows is (2677, 11)


,host_is_superhost,host_response_rate,review_scores_communication,host_has_profile_pic,host_identity_verified,host_location,host_about_bool,host_response_time_int
0,False,96,10.0,True,True,2,True,2.0
1,True,98,10.0,True,True,2,True,1.0
2,False,67,10.0,True,True,2,True,2.0
6,True,100,10.0,True,True,2,True,1.0
7,True,100,10.0,True,True,2,True,1.0


## Crisp-DM Step 4) Modelling for Question 3

In [43]:
# For our first step in the modelling process for question 3, we must normalize the data coming in.
# As an example, a host response rate between 0 and 100 wouldn't compare well to a boolean True or False when looking at correlation without normalizing.

# Used min max scaler to normalize the df_super_clean dataframe as many of the values are booleans
norm = preprocessing.MinMaxScaler().fit_transform(df_super_clean)
normalized_df = pd.DataFrame(norm, columns=df_super_clean.columns)

# Segmenting out the X's (all columns in the normalized df except the host_is_superhost column)
# Y is the host_is_superhost column
X = normalized_df.iloc[:,1:]
y = normalized_df.iloc[:,0]  


Function "coeffCapture" written to create a linear regression model from normalized variable data and then provide an array with the correlation coefficients.

In [44]:

def coeffCapture(var_data_norm_x, var_data_norm_y, test_size, random_state):
    '''
    This function will take a normalized set of variable data from columns in a pandas data frame,
    create a linear regression model that is taught using the train test split method,
    that will then be used to create and score prediction capabilities while also outputting
    the correlation coefficients of the x variables on the y value we're trying to understand more about.

    input:
    var_data_norm_x: pandas dataframe data in columns that have been normalized for use in a regression model. This data is trying to predict the y value.
    var_data_norm_y: pandas dataframe data in a single column that has been normalized for use in a regression model. This data is the y value we are trying to predict.

    output:
    Text string with test and train R^2 values, as well as an array with the correlation coefficients from the linear regression.
    '''

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(var_data_norm_x, var_data_norm_y, test_size=test_size, random_state=random_state)

    # Train model, do not have to normalize within function because we already did it!
    linreg_model = LinearRegression()
    linreg_model.fit(X_train, y_train)

    # Predict using model
    y_predict_train = linreg_model.predict(X_train)
    y_predict = linreg_model.predict(X_test)

    r2_results_train= r2_score(y_train, y_predict_train)
    r2_results_test = r2_score(y_test, y_predict)

    print('Train R^2 = {}'.format(round(r2_results_train,2)))
    print('Test R^2 = {}'.format(round(r2_results_test,2)))
    print('Linear Regression Model coefficients captured in output variable!')

    return linreg_model.coef_

Running our modelling function

In [45]:
linregCoeff_superHost = coeffCapture(X, y, 0.25, 20)

Train R^2 = 0.09
Test R^2 = 0.07
Linear Regression Model coefficients captured in output variable!


## Crisp-DM Step 5) Data Evaluation for Question 3
As seen above... our R^2 values are pretty low, even for the training data set! This could imply that there is a lot of variance in our training data, or that we would need to consider a different model to try and predict a categorical classification (aka a 1 for super host or a 0 for not super host)')

For each of the variables we received a correlation value between -1 and 1.  
In this graph, a highly correlated factor would receive either a 1 (positively correlated) or -1 (negatively correlated).  
In our case, a positive 1 means the factor is directly related to being a Super Host.

In [46]:
fig_corr = go.Figure(go.Bar(
            x=linregCoeff_superHost,
            y=normalized_df.iloc[:,1:].columns.tolist(),
            orientation='h',
            textposition= "inside",
            texttemplate = "%{x:.02}"
)
)
fig_corr.update_layout(
                    title = ("Percent Correlation of Key Airbnb Metrics to Determine if a Host is a Super Host for {:,} Listed Hosts in the Seattle Area<br>{:,} were Super Hosts and {:,} were not Super Hosts <br>Data Timeframe between {} and {}".format(
                        df_super_clean.shape[0],
                        df_super_clean['host_is_superhost'][df_super_clean['host_is_superhost'] == True].count(),
                        df_super_clean['host_is_superhost'][df_super_clean['host_is_superhost'] == False].count(),
                        min(df_cal['date']),
                        max(df_cal['date'])
                        )
                        ),
                    xaxis_title="Correlation Value (1 or -1 being highly correlated)",
                    yaxis_title="Host Metrics from Airbnb",
                    # yaxis2_title="Median Price of Apartments",
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="right",
                        x=0.99
                        )
                    )

fig_corr.show()

## Crisp-DM Step 6) Final Information + Conclusion for Question 3
As the data shows, the Review Scores Communication metric was very highly tied to being declared a super host. This value was provided from 0 to 10, with 10 being the highest communication score. Additionally, Host Response Rate was our second highest positive correlation, so it makes sense that to become a super host you must be very responsive and have good communication with your guests.

Interestingly enough, Host Has Profile Pic received a negative correlation with being a super host. Shown below, all but 3 of the 2677 hosts had profile pictures so that column likely could have been omitted in the correlation analysis.

Checking why host_has_profile_pic would be such a negative correlation to being a super host.  
Turns out, nearly every host (super or not) has a profile picture, so it makes sense that it could be interpreted incorrectly as being related to a non super host.  
For a loop of the analysis it would be prudent to remove variables that are nearly unanimous like this one!

In [47]:
df_super_clean['host_has_profile_pic'].value_counts()


True     2674
False       3
Name: host_has_profile_pic, dtype: int64

## Crisp-DM Step 6) Overall Conclusions

Question 1)  
If you’re looking to charge high rental prices for your apartment, zip codes: 98119, 98199, 98104, 98121, and 98101 had the highest median daily rental prices.  
Question 2)  
Become an Airbnb “super host” if you want to bump your rental prices even higher for your apartment.  
In the five priciest zip codes listed above, an average median price increase of ~19.44% was seen when super hosts listed vs. non-super hosts.  
Question 3)  
Lastly, we dove into “what does it take to become a super host?”  
To become a super host some key factors included: having a high Airbnb communication review score, having a high host response rate, and living in or near the Seattle area.

Overall takeaway: A first-time Seattle Airbnb rental property investor would likely set themselves up for success if they can:  
- Buy an apartment in one of the 5 mentioned zipcodes
- Commit to becoming a "super host" by being responsive and communicative with guests, among other key factors.